In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Model Parameters
pixel_width = 28
pixel_height = 28
input_shape = (pixel_width, pixel_height, 1)
num_of_classes = 10
batch_size = 64  # Reasonable batch size for stability
epochs = 13  # Keeping epochs smaller for quicker training
learning_rate = 0.001  # Base learning rate
dropout_rate = 0.4  # Adjusted dropout rate to prevent overfitting

# Load MNIST Dataset
(features_train, labels_train), (features_test, labels_test) = keras.datasets.mnist.load_data()

# Reshape and Normalize Data
features_train = features_train.reshape(-1, pixel_width, pixel_height, 1).astype("float32") / 255.0
features_test = features_test.reshape(-1, pixel_width, pixel_height, 1).astype("float32") / 255.0

# One-hot encode labels
labels_train = keras.utils.to_categorical(labels_train, num_of_classes)
labels_test = keras.utils.to_categorical(labels_test, num_of_classes)

# 🏆 **Data Augmentation**: Slightly increased range for robustness
datagen = ImageDataGenerator(
    rotation_range=20,  # Increase rotation range for more variability
    zoom_range=0.25,  # Slightly higher zoom range
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.15,  # Shear transformation
)
datagen.fit(features_train)

# 🧠 **Deep CNN Model with Enhanced Regularization**
model = Sequential()

# First Conv Block
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.0005)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(dropout_rate))

# Second Conv Block
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0005)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(dropout_rate))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.0005)))
model.add(Dropout(dropout_rate))  # Apply dropout in the FC layer
model.add(Dense(num_of_classes, activation='softmax'))

# **Optimizer with Learning Rate Decay**
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 🧠 **Advanced Callbacks**: Patience for early stopping and reduce LR
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1)

# 🚀 **Train Model with Augmented Data**
model.fit(datagen.flow(features_train, labels_train, batch_size=batch_size),
          validation_data=(features_test, labels_test),
          epochs=epochs,
          verbose=1,
          callbacks=[early_stopping, reduce_lr])

# 🎯 **Evaluate Model**
score = model.evaluate(features_test, labels_test, verbose=0)
print(f"🔥 Final Loss: {score[0]:.5f}, Accuracy: {score[1] * 100:.2f}%")

# Save the trained model
model.save('model.h5')  # Save model to a file
print("Model saved as 'model.h5'")



C:\ProgramData\anaconda3\envs\mnist_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\envs\mnist_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/13
938/938 ━━━━━━━━━━━━━━━━━━━━ 146s 153ms/step - accuracy: 0.4889 - loss: 1.9805 - val_accuracy: 0.9674 - val_loss: 0.3951 - learning_rate: 0.0010
Epoch 2/13
938/938 ━━━━━━━━━━━━━━━━━━━━ 158s 169ms/step - accuracy: 0.8682 - loss: 0.7379 - val_accuracy: 0.9764 - val_loss: 0.4079 - learning_rate: 0.0010
Epoch 3/13
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.9044 - loss: 0.6582  
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
938/938 ━━━━━━━━━━━━━━━━━━━━ 152s 162ms/step - accuracy: 0.9044 - loss: 0.6582 - val_accuracy: 0.9660 - val_loss: 0.5023 - learning_rate: 0.0010
Epoch 4/13
938/938 ━━━━━━━━━━━━━━━━━━━━ 147s 156ms/step - accuracy: 0.9353 - loss: 0.5683 - val_accuracy: 0.9852 - val_loss: 0.3384 - learning_rate: 5.0000e-04
Epoch 5/13
938/938 ━━━━━━━━━━━━━━━━━━━━ 145s 155ms/step - accuracy: 0.9476 - loss: 0.4671 - val_accuracy: 0.9872 - val_loss: 0.3082 - learning_rate: 5.0000e-04
Epoch 6/13
938/938 ━━━━━━━━━━━━━━━━━━━━ 147s 156ms/step -

🔥 Final Loss: 0.25611, Accuracy: 99.23%
Model saved as 'model.h5'
